In [ ]:

from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted!")
print("📁 Your Drive files are now in: /content/drive/MyDrive/")


In [ ]:
import logging
from pathlib import Path
from typing import Dict, Tuple, Any, Optional, List
import json
import os
import cv2
import numpy as np
from pathlib import Path
import logging
from tqdm import tqdm 
from datetime import datetime
import shutil
import csv
import pandas as pd
from sklearn.model_selection import train_test_split



# Setup logging (Senior-level: Proper logging instead of print)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('/content/data_labeling.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


# OPTION 1: Data in Google Drive (most common)
USE_DRIVE = True  # Set to False if data is uploaded to Colab

if USE_DRIVE:
    # Paths for Google Drive
    DRIVE_BASE = Path("/content/drive/MyDrive")
    # Update this path to match your Drive folder structure
    # Example: DRIVE_BASE / "ai-banana-earlystage" / "Data" / "Sigatoka pics"
    DATA_DIR = DRIVE_BASE / "Machine_Learning" / "Sigatoka pics"
    OUTPUT_DIR = DRIVE_BASE / "Machine_Learning" / "Data Labeling" 
    
    print("📁 Using Google Drive paths")
    print(f"   Data: {DATA_DIR}")
    print(f"   Output: {OUTPUT_DIR}")
else:
    # OPTION 2: Data uploaded to Colab (use /content/)
    DATA_DIR = Path("/content/Data/Sigatoka pics")
    OUTPUT_DIR = Path("/content/Data Labeling")
    
    print("📁 Using Colab workspace paths")
    print(f"   Data: {DATA_DIR}")
    print(f"   Output: {OUTPUT_DIR}")

# Validate paths exist
if not DATA_DIR.exists():
    logger.warning(f"⚠️  Data directory not found: {DATA_DIR}")
    print(f"\n⚠️  WARNING: Data directory not found!")
    print(f"   Please check the path: {DATA_DIR}")
    print(f"   Update DATA_DIR in this cell to match your folder structure")
else:
    print(f"✅ Data directory found: {DATA_DIR}")
    
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
logger.info(f"Output directory: {OUTPUT_DIR}")
print(f"✅ Output directory ready: {OUTPUT_DIR}")

# Class labels mapping
CLASS_LABELS = {
    "Stage1": "Stage1",
    "Stage2": "Stage2", 
    "Stage3": "Stage3",
    "Healthy": "Healthy"  # If you have healthy samples
}

# Image quality thresholds
QUALITY_THRESHOLDS = {
    "min_resolution": (256, 256),  # Minimum width, height
    "max_blur_threshold": 100.0,  # Laplacian variance threshold (lower = blurrier)
    "min_brightness": 20,  # Minimum average brightness (0-255)
    "max_brightness": 240,  # Maximum average brightness (0-255)
    "min_file_size_kb": 50,  # Minimum file size in KB
    "max_file_size_mb": 10,  # Maximum file size in MB
}

# Supported image formats
SUPPORTED_FORMATS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

# Metadata fields to extract
METADATA_FIELDS = [
    "image_path",
    "class_label",
    "width",
    "height",
    "file_size_kb",
    "blur_score",
    "brightness_score",
    "quality_status",
    "timestamp",
    "source_folder"
]

print("✓ Configuration loaded")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")


In [ ]:

logger = logging.getLogger(__name__)
class ImageQualityAssessor:

    
    def __init__(self, 
                 min_resolution: Tuple[int, int] = (256, 256),
                 max_blur_threshold: float = 100.0,
                 min_brightness: int = 20,
                 max_brightness: int = 240):
        """
        Initialize quality assessor with thresholds.
        
        Args:
            min_resolution: Minimum (width, height) in pixels
            max_blur_threshold: Laplacian variance threshold (lower = blurrier)
            min_brightness: Minimum average brightness (0-255)
            max_brightness: Maximum average brightness (0-255)
            
        Raises:
            ValueError: If thresholds are invalid
        """
        # Input validation (Senior-level)
        if min_resolution[0] < 0 or min_resolution[1] < 0:
            raise ValueError("Resolution must be positive")
        if max_blur_threshold < 0:
            raise ValueError("Blur threshold must be positive")
        if not (0 <= min_brightness < max_brightness <= 255):
            raise ValueError("Brightness thresholds must be in range [0, 255]")
        
        self.min_resolution = min_resolution
        self.max_blur_threshold = max_blur_threshold
        self.min_brightness = min_brightness
        self.max_brightness = max_brightness
        
        logger.info(f"ImageQualityAssessor initialized with thresholds: {self.__dict__}")
    
    def check_blur(self, image: np.ndarray) -> Dict[str, Any]:
        """
        Assess image blur using Laplacian variance.
        Args:
            image: Input image as numpy array
            
        Returns:
            Dict with 'score' (higher = sharper) and 'status' ('good'/'blurry'/'invalid')
        """
        if image is None or not isinstance(image, np.ndarray) or image.size == 0:
            logger.warning("Invalid image provided for blur check")
            return {"score": 0.0, "status": "invalid"}
        
        try:
            # Convert to grayscale if needed (Senior-level: Handle different formats)
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            elif len(image.shape) == 2:
                gray = image
            else:
                logger.warning(f"Unexpected image shape: {image.shape}")
                return {"score": 0.0, "status": "invalid"}
            
            # Calculate Laplacian variance (Senior-level: Efficient computation)
            laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
            status = "good" if laplacian_var >= self.max_blur_threshold else "blurry"
            
            return {"score": float(laplacian_var), "status": status}
        except Exception as e:
            logger.error(f"Error in blur check: {e}", exc_info=True)
            return {"score": 0.0, "status": "error", "error": str(e)}
    
    def check_brightness(self, image: np.ndarray) -> Dict[str, float]:
        """Assess image brightness"""
        if image is None or image.size == 0:
            return {"score": 0.0, "status": "invalid"}
        
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        avg_brightness = np.mean(gray)
        
        if avg_brightness < self.min_brightness:
            status = "too_dark"
        elif avg_brightness > self.max_brightness:
            status = "too_bright"
        else:
            status = "good"
        
        return {"score": float(avg_brightness), "status": status}
    
    def check_resolution(self, image: np.ndarray) -> Dict[str, Any]:
        """
        Check image resolution.
        Args:
            image: Input image as numpy array    
        Returns:
            Dict with width, height, and status
        """
        if image is None or not isinstance(image, np.ndarray) or image.size == 0:
            return {"width": 0, "height": 0, "status": "invalid"}
        
        try:
            height, width = image.shape[:2]
            min_w, min_h = self.min_resolution
            status = "good" if width >= min_w and height >= min_h else "too_small"
            
            return {"width": int(width), "height": int(height), "status": status}
        except Exception as e:
            logger.error(f"Error in resolution check: {e}", exc_info=True)
            return {"width": 0, "height": 0, "status": "error", "error": str(e)}
    
    def check_file_size(self, file_path: Path) -> Dict[str, Any]:
        """
        Check file size with proper error handling.
        Args:
            file_path: Path to file 
        Returns:
            Dict with file size information
        """
        if not isinstance(file_path, Path):
            file_path = Path(file_path)
        
        try:
            if not file_path.exists():
                logger.warning(f"File not found: {file_path}")
                return {"size_bytes": 0, "size_kb": 0.0, "size_mb": 0.0, "status": "file_not_found"}
            
            size_bytes = os.path.getsize(file_path)
            size_kb = size_bytes / 1024
            size_mb = size_kb / 1024
            
            if size_kb < 50:
                status = "too_small"
            elif size_mb > 10:
                status = "too_large"
            else:
                status = "good"
            
            return {
                "size_bytes": int(size_bytes),
                "size_kb": round(size_kb, 2),
                "size_mb": round(size_mb, 2),
                "status": status
            }
        except OSError as e:
            logger.error(f"OS error checking file size for {file_path}: {e}")
            return {"size_bytes": 0, "size_kb": 0.0, "size_mb": 0.0, "status": f"error: {str(e)}"}
        except Exception as e:
            logger.error(f"Unexpected error checking file size: {e}", exc_info=True)
            return {"size_bytes": 0, "size_kb": 0.0, "size_mb": 0.0, "status": f"error: {str(e)}"}
    
    def assess_image(self, image_path: Path) -> Dict[str, Any]:
        """
        Args:
            image_path: Path to image file
            
        Returns:
            Dict with comprehensive quality metrics
        """
        if not isinstance(image_path, Path):
            image_path = Path(image_path)
        
        results = {
            "image_path": str(image_path),
            "file_exists": False,
            "readable": False,
            "overall_status": "unknown"
        }
        
        if not image_path.exists():
            logger.warning(f"Image file not found: {image_path}")
            results["overall_status"] = "file_not_found"
            return results
        
        results["file_exists"] = True
        file_size_info = self.check_file_size(image_path)
        results.update(file_size_info)
        
        try:
            # Senior-level: Use cv2.IMREAD_UNCHANGED to preserve image properties
            image = cv2.imread(str(image_path), cv2.IMREAD_UNCHANGED)
            if image is None:
                logger.warning(f"Cannot read image (may be corrupted): {image_path}")
                results["overall_status"] = "cannot_read"
                return results
            
            results["readable"] = True
            
            # Perform all quality checks
            resolution_info = self.check_resolution(image)
            results.update(resolution_info)
            
            blur_info = self.check_blur(image)
            results["blur_score"] = blur_info.get("score", 0.0)
            results["blur_status"] = blur_info.get("status", "unknown")
            
            brightness_info = self.check_brightness(image)
            results["brightness_score"] = brightness_info.get("score", 0.0)
            results["brightness_status"] = brightness_info.get("status", "unknown")
            
            # Aggregate issues (Senior-level: Structured issue tracking)
            issues = []
            if resolution_info.get("status") != "good":
                issues.append("resolution")
            if blur_info.get("status") not in ["good", "unknown"]:
                issues.append("blur")
            if brightness_info.get("status") not in ["good", "unknown"]:
                issues.append("brightness")
            if file_size_info.get("status") != "good":
                issues.append("file_size")
            
            if not issues:
                results["overall_status"] = "good"
            else:
                results["overall_status"] = f"issues: {', '.join(issues)}"
                results["issues"] = issues
            
            logger.debug(f"Quality assessment complete for {image_path.name}: {results['overall_status']}")
            
        except cv2.error as e:
            logger.error(f"OpenCV error processing {image_path}: {e}", exc_info=True)
            results["overall_status"] = f"opencv_error: {str(e)}"
            results["error"] = str(e)
        except Exception as e:
            logger.error(f"Unexpected error assessing {image_path}: {e}", exc_info=True)
            results["overall_status"] = f"error: {str(e)}"
            results["error"] = str(e)
        
        return results

logger.info("✓ ImageQualityAssessor class loaded")


In [ ]:
logger = logging.getLogger(__name__)


class LabelingWorkflow:
    """Main workflow for creating labeled dataset"""
    
    def __init__(self, 
                 data_dir: Path,
                 output_dir: Path,
                 class_labels: Dict[str, str] = None,
                 quality_thresholds: Dict = None):
        self.data_dir = Path(data_dir)
        self.output_dir = Path(output_dir)
        self.class_labels = class_labels or CLASS_LABELS
        quality_thresholds = quality_thresholds or QUALITY_THRESHOLDS
        self.quality_assessor = ImageQualityAssessor(**quality_thresholds)
        
        # Create output structure
        self.output_dir.mkdir(parents=True, exist_ok=True)
        (self.output_dir / "images").mkdir(exist_ok=True)
        (self.output_dir / "labels").mkdir(exist_ok=True)
        (self.output_dir / "metadata").mkdir(exist_ok=True)
    
    def extract_class_from_folder(self, folder_path: Path) -> Optional[str]:
        """Extract class label from folder name"""
        folder_name = folder_path.name
        for key, label in self.class_labels.items():
            if key.lower() in folder_name.lower():
                return label
        return None
    
    def extract_metadata(self, image_path: Path, class_label: str) -> Dict[str, Any]:
        """
        Extract comprehensive metadata for a single image.
        
        Senior-level: Comprehensive metadata extraction with validation.
        
        Args:
            image_path: Path to image file
            class_label: Class label for the image
            
        Returns:
            Dict with comprehensive metadata
        """
        if not isinstance(image_path, Path):
            image_path = Path(image_path)
        
        metadata = {
            "image_path": str(image_path),
            "class_label": class_label,
            "timestamp": datetime.now().isoformat(),
            "source_folder": str(image_path.parent.name),
            "filename": image_path.name
        }
        
        # Quality assessment
        quality_result = self.quality_assessor.assess_image(image_path)
        
        metadata.update({
            "width": quality_result.get("width", 0),
            "height": quality_result.get("height", 0),
            "file_size_kb": round(quality_result.get("size_kb", 0), 2),
            "file_size_mb": round(quality_result.get("size_mb", 0), 3),
            "blur_score": round(quality_result.get("blur_score", 0), 2),
            "brightness_score": round(quality_result.get("brightness_score", 0), 2),
            "quality_status": quality_result.get("overall_status", "unknown"),
            "blur_status": quality_result.get("blur_status", "unknown"),
            "brightness_status": quality_result.get("brightness_status", "unknown")
        })
        
        # Senior-level: Add issues list if present
        if "issues" in quality_result:
            metadata["quality_issues"] = quality_result["issues"]
        
        return metadata
    
    def process_stage_folder(self, stage_folder: Path) -> List[Dict[str, any]]:
        """Process all images in a stage folder"""
        class_label = self.extract_class_from_folder(stage_folder)
        if not class_label:
            print(f"⚠ Warning: Could not determine class for {stage_folder}")
            return []
        
        print(f"\n📁 Processing {stage_folder.name} -> Class: {class_label}")
        
        image_files = []
        for ext in ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]:
            image_files.extend(stage_folder.glob(ext))
        
        if not image_files:
            print(f"  ⚠ No images found in {stage_folder}")
            return []
        
        metadata_list = []
        
        # Senior-level: Progress bar for batch processing
        for img_path in tqdm(image_files, desc=f"Processing {class_label}", leave=False):
            try:
                metadata = self.extract_metadata(img_path, class_label)
                output_image_name = f"{class_label}_{img_path.name}"
                output_image_path = self.output_dir / "images" / output_image_name
                
                # Senior-level: Check if file already exists to avoid overwriting
                if output_image_path.exists():
                    logger.warning(f"Output file already exists, skipping: {output_image_path}")
                    metadata["output_image_path"] = str(output_image_path)
                    metadata["output_image_name"] = output_image_name
                    metadata["status"] = "skipped_duplicate"
                    metadata_list.append(metadata)
                    continue
                
                shutil.copy2(img_path, output_image_path)
                metadata["output_image_path"] = str(output_image_path)
                metadata["output_image_name"] = output_image_name
                metadata["status"] = "success"
                metadata_list.append(metadata)
                logger.debug(f"Processed: {img_path.name} -> {output_image_name}")
                
            except shutil.Error as e:
                logger.error(f"File operation error for {img_path.name}: {e}")
                metadata["error"] = str(e)
                metadata["status"] = "error"
                metadata_list.append(metadata)
            except Exception as e:
                logger.error(f"Unexpected error processing {img_path.name}: {e}", exc_info=True)
                metadata = {"image_path": str(img_path), "class_label": class_label, "error": str(e), "status": "error"}
                metadata_list.append(metadata)
        
        return metadata_list
    
    def generate_csv_labels(self, metadata_list: List[Dict[str, any]]) -> Path:
        """Generate CSV file with labels"""
        csv_path = self.output_dir / "labels" / "dataset_labels.csv"
        
        fieldnames = METADATA_FIELDS.copy()
        fieldnames.extend(["output_image_path", "output_image_name"])
        
        with open(csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames, extrasaction='ignore')
            writer.writeheader()
            
            for metadata in metadata_list:
                row = {k: v for k, v in metadata.items() if k in fieldnames}
                writer.writerow(row)
        
        print(f"\n✓ CSV labels saved: {csv_path}")
        return csv_path
    
    def generate_json_labels(self, metadata_list: List[Dict[str, any]]) -> Path:
        """Generate JSON file with labels"""
        json_path = self.output_dir / "labels" / "dataset_labels.json"
        
        output_data = {
            "dataset_info": {
                "total_images": len(metadata_list),
                "classes": list(set(m["class_label"] for m in metadata_list)),
                "generated_at": datetime.now().isoformat()
            },
            "images": metadata_list
        }
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)
        
        print(f"✓ JSON labels saved: {json_path}")
        return json_path
    
    def run_workflow(self, 
                    generate_csv: bool = True,
                    generate_json: bool = True) -> Dict[str, any]:
        """Run complete labeling workflow"""
        print("="*60)
        print("BANANA LEAF DATA LABELING WORKFLOW")
        print("="*60)
        
        stage_folders = [d for d in self.data_dir.iterdir() 
                        if d.is_dir() and any(key.lower() in d.name.lower() 
                                            for key in self.class_labels.keys())]
        
        if not stage_folders:
            print(f"⚠ No stage folders found in {self.data_dir}")
            return {"error": "No stage folders found"}
        
        print(f"\n📂 Found {len(stage_folders)} stage folders")
        
        all_metadata = []
        for stage_folder in sorted(stage_folders):
            metadata_list = self.process_stage_folder(stage_folder)
            all_metadata.extend(metadata_list)
        
        if not all_metadata:
            print("\n⚠ No images processed")
            return {"error": "No images processed"}
        
        output_files = {}
        
        if generate_csv:
            csv_path = self.generate_csv_labels(all_metadata)
            output_files["csv"] = str(csv_path)
        
        if generate_json:
            json_path = self.generate_json_labels(all_metadata)
            output_files["json"] = str(json_path)
        
        class_counts = {}
        for metadata in all_metadata:
            class_label = metadata.get("class_label", "unknown")
            class_counts[class_label] = class_counts.get(class_label, 0) + 1
        
        summary = {
            "total_images": len(all_metadata),
            "class_distribution": class_counts,
            "output_files": output_files,
            "output_directory": str(self.output_dir)
        }
        
        print("\n" + "="*60)
        print("WORKFLOW SUMMARY")
        print("="*60)
        print(f"Total images processed: {summary['total_images']}")
        print(f"\nClass distribution:")
        for class_label, count in sorted(class_counts.items()):
            print(f"  {class_label}: {count}")
        print(f"\nOutput directory: {self.output_dir}")
        print("="*60)
        
        return summary

print("✓ LabelingWorkflow class loaded")

In [ ]:
valid_params = {
    'min_resolution': QUALITY_THRESHOLDS.get('min_resolution', (256, 256)),
    'max_blur_threshold': QUALITY_THRESHOLDS.get('max_blur_threshold', 100.0),
    'min_brightness': QUALITY_THRESHOLDS.get('min_brightness', 20),
    'max_brightness': QUALITY_THRESHOLDS.get('max_brightness', 240)
}
assessor = ImageQualityAssessor(**valid_params) 

# Example: Assess Stage1 images
stage1_dir = DATA_DIR / "Stage1"
if stage1_dir.exists():
    image_files = list(stage1_dir.glob("*.jpg"))[:5]  # First 5 images
    
    print("Quality Assessment Results:")
    for img_path in image_files:
        result = assessor.assess_image(img_path)
        print(f"\n{img_path.name}:")
        print(f"  Blur Score: {result.get('blur_score', 0):.2f} ({result.get('blur_status')})")
        print(f"  Brightness: {result.get('brightness_score', 0):.2f} ({result.get('brightness_status')})")
        print(f"  Resolution: {result.get('width')}x{result.get('height')}")
        print(f"  Status: {result.get('overall_status')}")

In [ ]:
try:
    # Filter QUALITY_THRESHOLDS to only include parameters expected by ImageQualityAssessor
    quality_params_for_assessor = {
        'min_resolution': QUALITY_THRESHOLDS.get('min_resolution'),
        'max_blur_threshold': QUALITY_THRESHOLDS.get('max_blur_threshold'),
        'min_brightness': QUALITY_THRESHOLDS.get('min_brightness'),
        'max_brightness': QUALITY_THRESHOLDS.get('max_brightness')
    }
    quality_params_for_assessor = {k: v for k, v in quality_params_for_assessor.items() if v is not None}

    workflow = LabelingWorkflow(
        data_dir=DATA_DIR,
        output_dir=OUTPUT_DIR,
        quality_thresholds=quality_params_for_assessor # Pass the filtered parameters
    )

    logger.info("Starting labeling workflow...")
    results = workflow.run_workflow(
        generate_csv=True,
        generate_json=True
    )

    if "error" in results:
        logger.error(f"Workflow failed: {results['error']}")
    else:
        logger.info(f"Workflow completed successfully. Processed {results.get('total_images', 0)} images.")

except Exception as e:
    logger.error(f"Fatal error in workflow: {e}", exc_info=True)

In [ ]:
if "error" not in results:
    # Load CSV to display
    csv_path = Path(results["output_files"]["csv"])
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        
        print("\n" + "="*60)
        print("DATASET SUMMARY ")
        print("="*60)
        
        print(f"\n📊 Total Images: {len(df)}")
        
        print(f"\n📈 Class Distribution:")
        class_dist = df["class_label"].value_counts()
        print(class_dist)
        print(f"   Balance ratio: {class_dist.max() / class_dist.min():.2f}x")
        
        print(f"\n✅ Quality Status Distribution:")
        quality_dist = df["quality_status"].value_counts()
        print(quality_dist)
        
        # Senior-level: Statistical summary
        print(f"\n📐 Image Statistics:")
        print(f"   Average Resolution: {df['width'].mean():.0f}x{df['height'].mean():.0f}")
        print(f"   Average Blur Score: {df['blur_score'].mean():.2f}")
        print(f"   Average Brightness: {df['brightness_score'].mean():.2f}")
        print(f"   Average File Size: {df['file_size_kb'].mean():.2f} KB")
        
        # Senior-level: Quality issues analysis
        if "quality_issues" in df.columns:
            print(f"\n⚠️  Quality Issues:")
            issue_counts = df["quality_issues"].value_counts()
            print(issue_counts)
        
        print(f"\n📋 Sample Data (first 5 rows):")
        display_cols = ["output_image_name", "class_label", "width", "height", 
                       "blur_score", "brightness_score", "quality_status"]
        available_cols = [col for col in display_cols if col in df.columns]
        print(df[available_cols].head().to_string())
        
        print(f"\n✓ Labels saved to: {csv_path}")
        print(f"✓ JSON saved to: {results['output_files']['json']}")
        print(f"✓ Log file: data_labeling.log")
        
        logger.info("Summary displayed successfully")
else:
    error_msg = results.get('error', 'Unknown error')
    logger.error(f"Workflow error: {error_msg}")
    print(f"❌ Error: {error_msg}")

In [ ]:
FINAL_DATASET_DIR = OUTPUT_DIR / 'final_dataset'
IMAGE_SOURCE_DIR = OUTPUT_DIR / 'images'

# Create the final_dataset directory if it doesn't exist
FINAL_DATASET_DIR.mkdir(parents=True, exist_ok=True)

# Load the dataset labels into a pandas DataFrame
labels_df = pd.read_csv(results['output_files']['csv'])

print(f"✓ Imported train_test_split.")
print(f"✓ Final dataset directory defined: {FINAL_DATASET_DIR}")
print(f"✓ Image source directory defined: {IMAGE_SOURCE_DIR}")
print(f"✓ Labels DataFrame loaded with {len(labels_df)} entries.")
labels_df.head()

In [ ]:
random_state = 42

# 1. Split labels_df into training and temporary combined validation/test set
train_df, val_test_df = train_test_split(
    labels_df,
    test_size=0.3,
    random_state=random_state,
    stratify=labels_df['class_label']
)

# 2. Further split val_test_df into validation and test sets
val_df, test_df = train_test_split(
    val_test_df,
    test_size=0.5,
    random_state=random_state,
    stratify=val_test_df['class_label']
)

# 3. Print the number of samples in each resulting DataFrame
print(f"\nDataset split summary:")
print(f"  Training set: {len(train_df)} samples")
print(f"  Validation set: {len(val_df)} samples")
print(f"  Test set: {len(test_df)} samples")

print(f"\nClass distribution in Training set:\n{train_df['class_label'].value_counts()}\n")
print(f"Class distribution in Validation set:\n{val_df['class_label'].value_counts()}\n")
print(f"Class distribution in Test set:\n{test_df['class_label'].value_counts()}\n")

In [ ]:
split_types = ['train', 'validation', 'test']

for split_type in split_types:
    split_dir = FINAL_DATASET_DIR / split_type
    split_dir.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created directory: {split_dir}")

    # Create 'images' and 'labels' subdirectories within each split
    images_dir = split_dir / 'images'
    images_dir.mkdir(parents=True, exist_ok=True)
    print(f"  ✓ Created directory: {images_dir}")

    labels_dir = split_dir / 'labels'
    labels_dir.mkdir(parents=True, exist_ok=True)
    print(f"  ✓ Created directory: {labels_dir}")

print("\n✓ Adjusted dataset directory structure created successfully.")

In [ ]:
splits_to_process = [
    (train_df, 'train'),
    (val_df, 'validation'),
    (test_df, 'test')
]

for df_split, split_type in splits_to_process:
    print(f"\nCopying images for {split_type} split...")
    for index, row in tqdm(df_split.iterrows(), total=len(df_split), desc=f"Copying {split_type} images"):
        source_image_name = row['output_image_name']
        # The class_label is implicitly part of the structure, but not needed for the image copy destination itself in this new structure

        source_path = IMAGE_SOURCE_DIR / source_image_name
        destination_path = FINAL_DATASET_DIR / split_type / 'images' / source_image_name

        try:
            shutil.copy2(source_path, destination_path)
        except FileNotFoundError:
            print(f"Warning: Source file not found: {source_path}")
        except Exception as e:
            print(f"Error copying {source_image_name} to {destination_path}: {e}")
    print(f"✓ Finished copying images for {split_type} split.")

print("\n✓ All images copied to their respective split directories.")

In [ ]:
split_dataframes = [
    (train_df, 'train'),
    (val_df, 'validation'),
    (test_df, 'test')
]

for df, split_type in split_dataframes:
    output_csv_path = FINAL_DATASET_DIR / split_type / 'labels' / f'{split_type}_labels.csv'
    df.to_csv(output_csv_path, index=False)
    print(f"✓ {split_type.capitalize()} labels saved to: {output_csv_path}")

print("\n✓ All split label CSVs generated.")

YOLO_CLASSIFICATION_DIR = OUTPUT_DIR / 'yolo_classification_dataset'
# Create YOLO classification structure
yolo_train_dir = YOLO_CLASSIFICATION_DIR / 'train'
yolo_val_dir = YOLO_CLASSIFICATION_DIR / 'validation'
yolo_test_dir = YOLO_CLASSIFICATION_DIR / 'test'

for split_dir in [yolo_train_dir, yolo_val_dir, yolo_test_dir]:
    split_dir.mkdir(parents=True, exist_ok=True)

print(f"\n YOLO classification dataset directory: {YOLO_CLASSIFICATION_DIR}")  
train_csv = FINAL_DATASET_DIR / 'train' / 'labels' / 'train_labels.csv'
val_csv = FINAL_DATASET_DIR / 'validation' / 'labels' / 'validation_labels.csv'
test_csv = FINAL_DATASET_DIR / 'test' / 'labels' / 'test_labels.csv'

print(f"\n Loading split CSVs...")
print(f"   Train: {train_csv.exists()}")
print(f"   Validation: {val_csv.exists()}")
print(f"   Test: {test_csv.exists()}")


if not all([train_csv.exists(), val_csv.exists(), test_csv.exists()]):
    print("\n ERROR: Split CSVs not found!")
    print("   Please run Cells 7-11 (data splitting) first")
else:
    print("All split CSVs found!")


In [ ]:

# Define splits mapping
splits = {
    'train': (train_csv, yolo_train_dir),
    'validation': (val_csv, yolo_val_dir),
    'test': (test_csv, yolo_test_dir)
}

# Get all unique classes from all splits
all_classes = set()
for split_name, (csv_path, _) in splits.items():
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        all_classes.update(df['class_label'].unique())

# Sort classes for consistent mapping
# Order: Healthy, Stage1, Stage2, Stage3, Stage4, Stage5, Stage6
sorted_classes = sorted(list(all_classes))
class_to_id = {cls: idx for idx, cls in enumerate(sorted_classes)}
id_to_class = {idx: cls for cls, idx in class_to_id.items()}

print(f"\nClasses found: {sorted_classes}")
print(f"\nClass to ID mapping:")
for cls, idx in class_to_id.items():
    print(f"   {idx}: {cls}")

# Process each split
for split_name, (csv_path, yolo_dir) in splits.items():
    if not csv_path.exists():
        continue

    print(f" Processing {split_name.upper()} split...")
    
    
    df = pd.read_csv(csv_path)
    
    # Copy images and organize by class
    copied_count = 0
    skipped_count = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {split_name}"):
        try:
            # Get image path and class from CSV
            source_image_path = Path(row['output_image_path'])
            class_label = row['class_label']  # Healthy, Stage1, Stage2, etc.
            
            # Create class directory in YOLO format
            # YOLO classification: images organized by class name
            class_dir = yolo_dir / class_label
            class_dir.mkdir(parents=True, exist_ok=True)
            
            # Copy image to class directory
            if source_image_path.exists():
                dest_image_path = class_dir / source_image_path.name
                shutil.copy2(source_image_path, dest_image_path)
                copied_count += 1
            else:
                skipped_count += 1
                logger.warning(f"Image not found: {source_image_path}")
                
        except Exception as e:
            logger.error(f"Error processing {row.get('output_image_name', 'unknown')}: {e}")
            skipped_count += 1
    
    print(f"\n {split_name.capitalize()} split:")
    print(f"   Copied: {copied_count} images")
    print(f"   Skipped: {skipped_count} images")
    
    # Show class distribution
    class_counts = df['class_label'].value_counts()
    print(f"\n   Class distribution:")
    for cls, count in class_counts.items():
        print(f"      {cls}: {count}")

In [ ]:
# Create YOLO dataset.yaml file
yolo_yaml_path = YOLO_CLASSIFICATION_DIR / 'dataset.yaml'

# YOLO classification configuration
yolo_config = {
    'path': str(YOLO_CLASSIFICATION_DIR.absolute()),
    'train': 'train',
    'val': 'validation',
    'test': 'test',
    'names': id_to_class,  # {0: 'Healthy', 1: 'Stage1', 2: 'Stage2', ...}
    'nc': len(sorted_classes)  # Number of classes
}

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(yolo_config, f, default_flow_style=False, sort_keys=False)

print(f"\nYAML file created: {yolo_yaml_path}")
print(f"\nConfiguration:")
print(f"   Path: {yolo_config['path']}")
print(f"   Train: {yolo_config['train']}")
print(f"   Validation: {yolo_config['val']}")
print(f"   Test: {yolo_config['test']}")
print(f"   Number of classes: {yolo_config['nc']}")
print(f"   Classes: {list(yolo_config['names'].values())}")

with open(yolo_yaml_path, 'r') as f:
    print(f.read())

for split_name in ['train', 'validation', 'test']:
    split_dir = YOLO_CLASSIFICATION_DIR / split_name
    if split_dir.exists():
        print(f"\n📂 {split_name.upper()}:")
        class_dirs = [d for d in split_dir.iterdir() if d.is_dir()]
        total_images = 0
        
        for class_dir in sorted(class_dirs):
            images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.JPG')) + \
                     list(class_dir.glob('*.png')) + list(class_dir.glob('*.PNG'))
            total_images += len(images)
            print(f"   {class_dir.name}: {len(images)} images")
        
        print(f"   Total: {total_images} images")

print(f"\n📁 Dataset location: {YOLO_CLASSIFICATION_DIR}")
print(f"   Config file: {yolo_yaml_path}")
print(f"\n Next step: Train YOLO classification model")
print(f"   Use: model = YOLO('yolov8n-cls.pt')  # or yolov8s-cls.pt, yolov8m-cls.pt")
print(f"   model.train(data='{yolo_yaml_path}', epochs=100)")
print(f"   Dataset config: {yolo_yaml_path}")
print(f"   Dataset location: {YOLO_CLASSIFICATION_DIR}")
